In [1]:
import subprocess
import time
import json
import matplotlib.pyplot as plt 
import csv 
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns  
import pickle
from itertools import islice
from scipy.cluster.vq import kmeans
from scipy.cluster.vq import vq

In [2]:
#fee_per_vbyte_block_csv = open('/home/oscar/jufobtc/code/fee_vybte_per_block_650000_657000.csv','r')
fee_per_vbyte_block_csv = open('/home/oscar/jufobtc/code/avg_fpvb_650000_700000.csv','r')
colnames = ['time_stamp', 'block_height', 'avg_fee_block']
df_avg_fee_block = pd.read_csv(fee_per_vbyte_block_csv,delimiter=' ', names=colnames)
#print(df_avg_fee_block.loc[df_avg_fee_block['block_height']<0])
#print(df_avg_fee_block)

In [49]:
delta_b = 6 #Blockintervall

df_avg_fee_block.sort_values(by='block_height')
df_avg_fee_block['block_time'] = df_avg_fee_block['time_stamp'].diff(periods=1)
df_avg_fee_block['block_time'] = df_avg_fee_block['block_time']/60 # in Minuten umrechnen
df_avg_fee_block['moving_average_fee'] = df_avg_fee_block['avg_fee_block'].rolling(144).mean()
df_avg_fee_block['change_fee'] = df_avg_fee_block['avg_fee_block'].diff()
df_avg_fee_block['change_trend'] = df_avg_fee_block['change_fee'].rolling(delta_b).sum()
#df_avg_fee_block['change_trend_ma'] = df_avg_fee_block['moving_average_fee'].rolling(24*6).sum()

df_avg_fee_block['anomaly'] = ((df_avg_fee_block['avg_fee_block']-df_avg_fee_block['moving_average_fee'])>150) | ((df_avg_fee_block['avg_fee_block']-df_avg_fee_block['moving_average_fee'])<-75)


print(df_avg_fee_block.loc[df_avg_fee_block['avg_fee_block'].isnull()==True])
#print(df_avg_fee_block.loc[df_avg_fee_block['anomaly']==True])

Empty DataFrame
Columns: [time_stamp, block_height, avg_fee_block, block_time, moving_average_fee, change_fee, change_trend, anomaly, median_conf_time_block, sign_change_trend, change_trend_6, sign_change_trend_6, change_trend_72]
Index: []


In [4]:
conf_time_csv = open('/home/oscar/jufobtc/code/test_comp_time.csv','r')
colnames = ['time_stamp', 'conf_time','block_height']
df_conf_time = pd.read_csv(conf_time_csv,delimiter=' ', names=colnames)
print('df loaded')
#print(df_conf_time)


df loaded


In [5]:
fee_per_tx_block_650000_659998_csv = open('/home/oscar/jufobtc/fee_block_650000_659998.csv','r')
df_tx_fee = pd.read_csv(fee_per_tx_block_650000_659998_csv,delimiter=' ', names=['tx_hash', 'tx_fee','block_height'])
print(df_tx_fee)

                                                    tx_hash      tx_fee  \
0         8143b3b341f665b22adcb8489158356c03f7c93cf4e4fa...    0.000000   
1         c17d8179d792928bb017d12654cd12c72aca35ca0cb258...  475.270769   
2         a795b27a52e74f07bf9def3edad50a06db902ee173d30b...  338.053097   
3         c1b7b41cc7bc40fe528a08528ae150e2fdab6aa8501dd1...  332.492147   
4         35064cb6846becca4594ccb45814a838039be1f08d3c04...  301.204819   
...                                                     ...         ...   
21359491  76daccf3b2067d63e6176f65d0cb4a576da19d83c3c479...    8.064343   
21359492  b2ceb2cf53b224917093c9d2639660976fdc2be3c7c92d...    8.064343   
21359493  5f376f04982af7066200a04ad4e490c1733055ffe8857b...    8.064343   
21359494  69e9cf4af9f34f21498942cf8e2fcb286b6527b2b4eeb7...    8.060150   
21359495  6d3c661fa221649e20e8f960e60231e691836452394189...    8.046243   

          block_height  
0               650000  
1               650000  
2               650000  

In [6]:
df_conf_time_fee_per_tx = pd.DataFrame(columns=['tx_hash','conf_time','tx_fee','block_height'])
df_conf_time_fee_per_tx['tx_hash'] = df_tx_fee['tx_hash']
df_conf_time_fee_per_tx['conf_time'] = df_conf_time['conf_time']
df_conf_time_fee_per_tx['tx_fee'] = df_tx_fee['tx_fee']
df_conf_time_fee_per_tx['conf_time'] = df_conf_time_fee_per_tx['conf_time']/60 # in Min umgerechnet
df_conf_time_fee_per_tx['block_height'] = df_tx_fee['block_height']
print(df_conf_time_fee_per_tx)

                                                    tx_hash  conf_time  \
0         8143b3b341f665b22adcb8489158356c03f7c93cf4e4fa...   3.517750   
1         c17d8179d792928bb017d12654cd12c72aca35ca0cb258...   1.911333   
2         a795b27a52e74f07bf9def3edad50a06db902ee173d30b...   2.376767   
3         c1b7b41cc7bc40fe528a08528ae150e2fdab6aa8501dd1...   1.422300   
4         35064cb6846becca4594ccb45814a838039be1f08d3c04...   2.149017   
...                                                     ...        ...   
21359491  76daccf3b2067d63e6176f65d0cb4a576da19d83c3c479...        NaN   
21359492  b2ceb2cf53b224917093c9d2639660976fdc2be3c7c92d...        NaN   
21359493  5f376f04982af7066200a04ad4e490c1733055ffe8857b...        NaN   
21359494  69e9cf4af9f34f21498942cf8e2fcb286b6527b2b4eeb7...        NaN   
21359495  6d3c661fa221649e20e8f960e60231e691836452394189...        NaN   

              tx_fee  block_height  
0           0.000000        650000  
1         475.270769        650000  


In [7]:
df_block_fee_conf_time = pd.DataFrame(columns=['conf_time','avg_fee_block','block_height'])
df_block_fee_conf_time['conf_time'],df_block_fee_conf_time['block_height'] = df_conf_time['conf_time']/60, df_conf_time['block_height']
#print(df_block_fee_conf_time)
#df_block_fee_conf_time['avg_fee_block'] = df_avg_fee_block['avg_fee_block'].loc[df_avg_fee_block['block_height']==df_block_fee_conf_time['block_height']]
#dict_avg_fee_block = df_avg_fee_block.set_index('block_height').T.to_dict('list')
df_avg_fee_block_two_row = pd.DataFrame(columns=['block_height','avg_fee_block'])
df_avg_fee_block_two_row['block_height'],df_avg_fee_block_two_row['avg_fee_block'] = df_avg_fee_block['block_height'], df_avg_fee_block['avg_fee_block']
dict_avg_fee_block = df_avg_fee_block_two_row.to_dict()
#df_block_fee_conf_time['avg_fee_block'] = df_avg_fee_block['avg_fee_block']
dict_avg_fee_block[650000] = 24.453589030036483
df_block_fee_conf_time['avg_fee_block'] = df_block_fee_conf_time['block_height'].map(dict_avg_fee_block)
df_block_fee_conf_time = pd.merge(df_block_fee_conf_time, df_avg_fee_block, on='block_height')
df_block_fee_conf_time['cleaned_conf_time'] = df_block_fee_conf_time['confavg_fee_block_time'] - df_block_fee_conf_time['block_time']




In [8]:
dict = {}

for block_height in range(650000,656000):
    data = df_block_fee_conf_time.loc[df_block_fee_conf_time['block_height']==block_height]
    dict[block_height] = data['cleaned_conf_time'].median()
    #df_block_fee_conf_time['median_conf_time_block'].loc[df_block_fee_conf_time['block_height']==block_height] = df_block_fee_conf_time['conf_time'].loc[df_block_fee_conf_time['block_height']==block_height].median()
df_avg_fee_block['median_conf_time_block'] = df_avg_fee_block['block_height'].map(dict)    
df_block_fee_conf_time['median_conf_time_block'] = df_block_fee_conf_time['block_height'].map(dict)    


In [9]:
k = 0
df_temp = df_block_fee_conf_time['avg_fee_block']
df_temp['sign_median_conf_time'] = 0
df_temp['sign_change_trend'] = 0
df_avg_fee_block['sign_change_trend'] = 0
df_temp['sign_median_conf_time'].loc[df_temp['median_conf_time_block']>0] = 'positive'
df_temp['sign_median_conf_time'].loc[df_temp['median_conf_time_block']<0] = 'negative'
df_temp['sign_change_trend'].loc[df_temp['change_trend']>k] = 'positive'
df_temp['sign_change_trend'].loc[df_temp['change_trend']<-k] = 'negative'
df_avg_fee_block['sign_change_trend'].loc[df_avg_fee_block['change_trend']>k] = 'positive'
df_avg_fee_block['sign_change_trend'].loc[df_avg_fee_block['change_trend']<-k] = 'negative'

/tmp/ipykernel_3375/3544074435.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['sign_median_conf_time'].loc[df_temp['median_conf_time_block']>0] = 'positive'
/tmp/ipykernel_3375/3544074435.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['sign_change_trend'].loc[df_temp['change_trend']>k] = 'positive'
/tmp/ipykernel_3375/3544074435.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_avg_fee_block['sign_change

In [ ]:

df_model = df_avg_fee_block

df_model['change_trend_6'] = df_model['change_fee'].rolling(6).sum()

df_model['sign_change_trend_6'] = 0
df_model['sign_change_trend_6'].loc[df_model['change_trend_6']>k] = 'positive'
df_model['sign_change_trend_6'].loc[df_model['change_trend_6']<-k] = 'negative'
print(df_model)

In [11]:


delta_b = 72
column = 'change_trend_' + str(delta_b)
df_model[column] = df_model['change_fee'].rolling(delta_b).sum()

print(len(df_model.index))
print(df_model.index)
print(df_model)

63051
RangeIndex(start=0, stop=63051, step=1)
       time_stamp  block_height  avg_fee_block  block_time  \
0      1601077461        650001      22.292135         NaN   
1      1601077784        650002      21.040189    5.383333   
2      1601078147        650003      33.012255    6.050000   
3      1601078753        650004      51.644981   10.100000   
4      1601078976        650005      20.168499    3.716667   
...           ...           ...            ...         ...   
63046  1638888574        713051      12.732729   15.700000   
63047  1638890508        713052      23.110041   32.233333   
63048  1638890665        713053       7.644869    2.616667   
63049  1638890773        713054       6.233872    1.800000   
63050  1638890975        713055       4.266280    3.366667   

       moving_average_fee  change_fee  change_trend  anomaly  \
0                     NaN         NaN           NaN    False   
1                     NaN   -1.251946           NaN    False   
2                

In [48]:
cnt_trend_true = 0
cnt_trend_false = 0
cnt_total = 0 
cnt_blocks = 0
delta_check = 1
k = 0

for block in range(650001,656000):
    data = df_model.loc[df_avg_fee_block['block_height']==block]
    data_next = df_model.loc[df_avg_fee_block['block_height']==block+delta_check]
    data_last = df_model.loc[df_avg_fee_block['block_height']==block-delta_check]
    #data = data.reset_index(drop=True)
    #data_next = data_next.reset_index(drop=True)
    if pd.isnull(data[column].values[0]): 
        cnt_blocks = cnt_blocks + 1
        continue
    if data[column].values[0] > k and data_last['moving_average_fee'].values[0] < data_next['moving_average_fee'].values[0]:
        cnt_trend_true = cnt_trend_true + 1
        cnt_total = cnt_total + 1
        
    elif data[column].values[0] < -k and data_last['moving_average_fee'].values[0] > data_next['moving_average_fee'].values[0]:
        cnt_trend_true = cnt_trend_true + 1
        cnt_total = cnt_total + 1
        
    else:
        cnt_trend_false = cnt_trend_false + 1
        cnt_total = cnt_total + 1
        
    if data[column].values[0] > -k and data[column].values[0] < k:
        cnt_trend_false = cnt_trend_false - 1
        cnt_total = cnt_total - 1
        
    cnt_blocks = cnt_blocks + 1

print('parameter: ', '\n k=', k, '\n Zeitfenster MS=', delta_b, '\n Zeitfenster Comparison ( block +/-)=' , delta_check  )    
print('count blocks: ', cnt_blocks)        
print('count richtig: ',cnt_trend_true,)
print('count falsch ', cnt_trend_false) 
print('rate: ', (cnt_trend_true/cnt_total)) 
print('count total comps : ', cnt_total)



parameter:  
 k= 0 
 Zeitfenster MS= 18 
 Zeitfenster Comparison ( block +/-)= 1
count blocks:  5999
count richtig:  3861
count falsch  2066
rate:  0.651425679095664
count total comps :  5927


In [51]:
cnt__total_trend_true = 0 #test mit block-x und block+x
cnt_total_trend_false = 0
cnt_growing_true = 0
cnt_falling_true = 0
cnt_growing_false = 0
cnt_falling_false = 0
cnt_total = 0 
cnt_blocks = 0
cnt_skipped = 0
delta_check = 1
k = -1
temp_data = df_avg_fee_block
metric_growth = 'avg_fee_block'
#temp_data.dropna(subset='median_conf_time_block')
#temp_data.set_index('block_height')

for block in range(650001,656000):
    data = df_model.loc[df_model['block_height']==block]
    data_next = df_model.loc[df_model['block_height']==block+delta_check]
    data_last = df_model.loc[df_model['block_height']==block-delta_check]
    #data = data.reset_index(drop=True)
    #data_next = data_next.reset_index(drop=True)
    #print(data['median_conf_time_block'],data['block_height'])
    try: 
        if data['median_conf_time_block'].values[0] > k and data_last['avg_fee_block'].values[0] < data_next['avg_fee_block'].values[0]:
            x = 0
    except:
        print(block)
        cnt_skipped = cnt_skipped + 1
        cnt_skipped = cnt_skipped + 1
        continue
    else:
        #if data['median_conf_time_block'].iloc[0].isnull() == True:
        #    cnt_skipped = cnt_skipped + 1
        #    continue 
        #print(block)
        #print(data['median_conf_time_block'].values)
        #print(data_last['avg_fee_block'].values)
        #print(data_next['avg_fee_block'].values)
        if pd.isnull(data['median_conf_time_block'].values[0]):
            cnt_skipped = cnt_skipped + 1
            cnt_blocks = cnt_blocks + 1
            continue
        if data['median_conf_time_block'].loc[data['block_height']==block].values == 0 or len(data_last['avg_fee_block'].values) == 0 or len(data_next['avg_fee_block'].values) == 0: 
            print(block)
            print(data['median_conf_time_block'].values)
            print(data_last['avg_fee_block'].values)
            print(data_next['avg_fee_block'].values)
            cnt_skipped = cnt_skipped + 1
            cnt_blocks = cnt_blocks + 1
            continue
        if data['median_conf_time_block'].loc[data['block_height']==block].values < k and data['moving_average_fee'].values < data_next['moving_average_fee'].values[0]:
            cnt_growing_true = cnt_growing_true + 1
            cnt_total = cnt_total + 1
            #print('fallend richtig')
            
        elif data['median_conf_time_block'].values[0] > k and data['moving_average_fee'].values > data_next['moving_average_fee'].values[0]:
            #print('steigend richtig')
            cnt_falling_true = cnt_falling_true + 1
            cnt_total = cnt_total + 1

        elif data['median_conf_time_block'].values[0] < k and data['moving_average_fee'].values > data_next['moving_average_fee'].values[0]:
            #print('falsch')
            cnt_growing_false = cnt_growing_false + 1
            cnt_total = cnt_total + 1
        
        elif data['median_conf_time_block'].values[0] > k and data['moving_average_fee'].values < data_next['moving_average_fee'].values[0]:
            #print('falsch')
            cnt_falling_false = cnt_falling_false + 1
            cnt_total = cnt_total + 1
        
        #if data[column].values[0] > -k and data[column].values[0] < k:
        #    cnt_trend_false = cnt_trend_false - 1
         #   cnt_total = cnt_total - 1
          
        cnt_blocks = cnt_blocks + 1
cnt_total_trend_true = cnt_falling_true + cnt_growing_true
cnt_total_trend_false = cnt_falling_false+cnt_growing_false
print('parameter: ', '\n k =', k, '\n Zeitfenster Comparison ( block +/-) =' , delta_check  )    
print('count blocks: ', cnt_blocks)        
print('count steigend erkannt: ',cnt_growing_true,)
print('count steigen nicht erkannt: ', cnt_growing_false)
print('count fallend erkannt: ',cnt_falling_true)
print('count fallend nicht erkannt :',cnt_falling_false)
print('count total erkannt ', str(cnt_total_trend_true)) 
print('count total nicht erkannt ', str(cnt_total_trend_false))
print('rate: ', (cnt_total_trend_true/cnt_total)) 
print('count skipped: ', cnt_skipped)
print('count total comps : ', cnt_total)

650002
650003
[-2.78096667]
[]
[25.44444367]
parameter:  
 k = -1 
 Zeitfenster Comparison ( block +/-) = 3
count blocks:  5998
count steigend erkannt:  1237
count steigen nicht erkannt:  1120
count fallend erkannt:  1122
count fallend nicht erkannt : 1154
count total erkannt  2359
count total nicht erkannt  2274
rate:  0.5091733218217138
count skipped:  1230
count total comps :  4633


In [52]:
cnt__total_trend_true = 0 #test mit block-b und block+b 
cnt_total_trend_false = 0
cnt_growing_true = 0
cnt_falling_true = 0
cnt_growing_false = 0
cnt_falling_false = 0
cnt_total = 0 
cnt_blocks = 0
cnt_skipped = 0
delta_check = 1 
k = -2.5
temp_data = df_avg_fee_block
#temp_data.dropna(subset='median_conf_time_block')
#temp_data.set_index('block_height')

for block in range(650001,656000):
    data = df_model.loc[df_model['block_height']==block]
    data_next = df_model.loc[df_model['block_height']==block+delta_check]
    data = df_model.loc[df_model['block_height']==block-delta_check]
    #data = data.reset_index(drop=True)
    #data_next = data_next.reset_index(drop=True)
    #print(data['median_conf_time_block'],data['block_height'])
    try: 
        if data['median_conf_time_block'].values[0] > k and data_last['avg_fee_block'].values[0] < data_next['avg_fee_block'].values[0]:
            x = 0
    except:
        print(block)
        cnt_skipped = cnt_skipped + 1
        cnt_skipped = cnt_skipped + 1
        continue
    else:
        #if data['median_conf_time_block'].iloc[0].isnull() == True:
        #    cnt_skipped = cnt_skipped + 1
        #    continue 
        #print(block)
        #print(data['median_conf_time_block'].values)
        #print(data_last['avg_fee_block'].values)
        #print(data_next['avg_fee_block'].values)
        if pd.isnull(data['median_conf_time_block'].values[0]):
            cnt_skipped = cnt_skipped + 1
            cnt_blocks = cnt_blocks + 1
            continue
        if data['median_conf_time_block'].loc[data['block_height']==block].values == 0 or len(data_last['avg_fee_block'].values) == 0 or len(data_next['avg_fee_block'].values) == 0: 
            print(block)
            print(data['median_conf_time_block'].values)
            print(data_last['avg_fee_block'].values)
            print(data_next['avg_fee_block'].values)
            cnt_skipped = cnt_skipped + 1
            cnt_blocks = cnt_blocks + 1
            continue
        if data['median_conf_time_block'].loc[data['block_height']==block].values < k and data['moving_average_fee'].values < data_next['moving_average_fee'].values[0]:
            cnt_growing_true = cnt_growing_true + 1
            cnt_total = cnt_total + 1
            #print('fallend richtig')
            
        elif data['median_conf_time_block'].values[0] > k and data['moving_average_fee'].values > data_next['moving_average_fee'].values[0]:
            #print('steigend richtig')
            cnt_falling_true = cnt_falling_true + 1
            cnt_total = cnt_total + 1

        elif data['median_conf_time_block'].values[0] < k and data['moving_average_fee'].values > data_next['moving_average_fee'].values[0]:
            #print('falsch')
            cnt_growing_false = cnt_growing_false + 1
            cnt_total = cnt_total + 1
        
        elif data['median_conf_time_block'].values[0] > k and data['moving_average_fee'].values < data_next['moving_average_fee'].values[0]:
            #print('falsch')
            cnt_falling_false = cnt_falling_false + 1
            cnt_total = cnt_total + 1
        
        #if data[column].values[0] > -k and data[column].values[0] < k:
        #    cnt_trend_false = cnt_trend_false - 1
         #   cnt_total = cnt_total - 1
          
        cnt_blocks = cnt_blocks + 1
cnt_total_trend_true = cnt_falling_true + cnt_growing_true
cnt_total_trend_false = cnt_falling_false+cnt_growing_false
print('parameter: ', '\n k =', k, '\n Zeitfenster Comparison ( block +/-) =' , delta_check  )    
print('count blocks: ', cnt_blocks)        
print('count steigend erkannt: ',cnt_growing_true,)
print('count steigen nicht erkannt: ', cnt_growing_false)
print('count fallend erkannt: ',cnt_falling_true)
print('count fallend nicht erkannt :',cnt_falling_false)
print('count total erkannt ', str(cnt_total_trend_true)) 
print('count total nicht erkannt ', str(cnt_total_trend_false))
print('rate: ', (cnt_total_trend_true/cnt_total)) 
print('count skipped: ', cnt_skipped)
print('count total comps : ', cnt_total)

650001


/tmp/ipykernel_3375/3819957617.py:43: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if data['median_conf_time_block'].loc[data['block_height']==block].values == 0 or len(data_last['avg_fee_block'].values) == 0 or len(data_next['avg_fee_block'].values) == 0:
/tmp/ipykernel_3375/3819957617.py:51: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if data['median_conf_time_block'].loc[data['block_height']==block].values < k and data['moving_average_fee'].values < data_next['moving_average_fee'].values[0]:


parameter:  
 k = -2.5 
 Zeitfenster Comparison ( block +/-) = 1
count blocks:  5998
count steigend erkannt:  0
count steigen nicht erkannt:  865
count fallend erkannt:  1355
count fallend nicht erkannt : 1365
count total erkannt  1355
count total nicht erkannt  2230
rate:  0.37796373779637377
count skipped:  1229
count total comps :  3585


In [33]:
cnt__total_trend_true = 0 #test mit aktuellem block vs "nächstem" block und avg_feeblock
cnt_total_trend_false = 0
cnt_growing_true = 0
cnt_falling_true = 0
cnt_growing_false = 0
cnt_falling_false = 0
cnt_total = 0 
cnt_blocks = 0
cnt_skipped = 0
delta_check = 3
k = -2,5
k1 = -5
k2 = 5
temp_data = df_avg_fee_block
#temp_data.dropna(subset='median_conf_time_block')
#temp_data.set_index('block_height')

for block in range(650001,656000):
    data = df_model.loc[df_model['block_height']==block]
    data_next = df_model.loc[df_model['block_height']==block+delta_check]
    data_last = df_model.loc[df_model['block_height']==block-delta_check]
    #data = data.reset_index(drop=True)
    #data_next = data_next.reset_index(drop=True)
    #print(data['median_conf_time_block'],data['block_height'])
    try: 
        if data['median_conf_time_block'].values[0] > k and data['avg_fee_block'].values[0] > data_last['avg_fee_block'].values[0] < data_next['avg_fee_block'].values[0]:
            x = 0
    except:
        print(block)
        cnt_skipped = cnt_skipped + 1
        cnt_skipped = cnt_skipped + 1
        cnt_blocks = cnt_blocks + 1
        continue
    else:
        #if data['median_conf_time_block'].iloc[0].isnull() == True:
        #    cnt_skipped = cnt_skipped + 1
        #    continue 
        #print(block)
        #print(data['median_conf_time_block'].values)
        #print(data_last['avg_fee_block'].values)
        #print(data_next['avg_fee_block'].values)
        if pd.isnull(data['median_conf_time_block'].values[0]):
            cnt_skipped = cnt_skipped + 1
            cnt_blocks = cnt_blocks + 1
            continue
        if data['median_conf_time_block'].loc[data['block_height']==block].values == 0 or len(data_last['avg_fee_block'].values) == 0 or len(data_next['avg_fee_block'].values) == 0: 
            print(block)
            print(data['median_conf_time_block'].values)
            print(data_last['avg_fee_block'].values)
            print(data_next['avg_fee_block'].values)
            cnt_skipped = cnt_skipped + 1
            cnt_blocks = cnt_blocks + 1
            continue
        if data['median_conf_time_block'].values[0] > k1 and data['median_conf_time_block'].values[0] < k2:
            cnt_skipped = cnt_skipped + 1 
            cnt_blocks = cnt_blocks + 1
            continue
        
        if data['median_conf_time_block'].values[0] < k1 and data['moving_average_fee'].values [0] < data_next['moving_average_fee'].values[0]:
            cnt_growing_true = cnt_growing_true + 1
            cnt_total = cnt_total + 1
            #print('fallend richtig')
            
        elif data['median_conf_time_block'].values[0] > k2 and data['moving_average_fee'].values[0] > data_next['moving_average_fee'].values[0]:
            #print('steigend richtig')
            cnt_falling_true = cnt_falling_true + 1
            cnt_total = cnt_total + 1

        elif data['median_conf_time_block'].values[0] < k1 and data['moving_average_fee'].values[0] > data_next['moving_average_fee'].values[0]:
            #print('falsch')
            cnt_growing_false = cnt_growing_false + 1
            cnt_total = cnt_total + 1
        
        elif data['median_conf_time_block'].values[0] > k2 and data['moving_average_fee'].values[0] < data_next['moving_average_fee'].values[0]:
            #print('falsch')
            cnt_falling_false = cnt_falling_false + 1
            cnt_total = cnt_total + 1
        
        #if data[column].values[0] > -k and data[column].values[0] < k:
        #    cnt_trend_false = cnt_trend_false - 1
         #   cnt_total = cnt_total - 1
     

          
        cnt_blocks = cnt_blocks + 1
cnt_total_trend_true = cnt_falling_true + cnt_growing_true
cnt_total_trend_false = cnt_falling_false+cnt_growing_false
print('parameter: ', '\n k =', k, '\n Zeitfenster Comparison ( block +/-) =' , delta_check  )    
print('count blocks: ', cnt_blocks)        
print('count steigend erkannt: ',cnt_growing_true,)
print('count fälschlicherweise steigend: ', cnt_growing_false)
print('count fallend erkannt: ',cnt_falling_true)
print('count fälschlicherweise fallend: ',cnt_falling_false)
print('count total erkannt ', str(cnt_total_trend_true)) 
print('count total nicht erkannt ', str(cnt_total_trend_false))
print('rate: ', (cnt_total_trend_true/cnt_total)) 
print('count skipped: ', cnt_skipped)
print('count total comps : ', cnt_total)

650002
[10.778875]
[]
[20.16849949]
650003
[-2.78096667]
[]
[25.44444367]
parameter:  
 k = 50 
 Zeitfenster Comparison ( block +/-) = 3
count blocks:  5999
count steigend erkannt:  717
count fälschlicherweise steigend:  589
count fallend erkannt:  842
count fälschlicherweise fallend:  891
count total erkannt  1559
count total nicht erkannt  1480
rate:  0.5129976966107272
count skipped:  2872
count total comps :  3039


In [16]:
df = df_model.loc[df_model['block_height']<656000]
print(df[df.change_fee > 0].count())
print(df[df.change_fee < 0].count())

time_stamp                2781
block_height              2781
avg_fee_block             2781
block_time                2781
moving_average_fee        2711
change_fee                2781
change_trend              2745
anomaly                   2781
median_conf_time_block    2206
sign_change_trend         2781
change_trend_6            2778
sign_change_trend_6       2781
change_trend_72           2745
dtype: int64
time_stamp                3216
block_height              3216
avg_fee_block             3216
block_time                3216
moving_average_fee        3144
change_fee                3216
change_trend              3181
anomaly                   3216
median_conf_time_block    2566
sign_change_trend         3216
change_trend_6            3214
sign_change_trend_6       3216
change_trend_72           3181
dtype: int64
